In [ ]:
import os
import pdb
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from argparse import ArgumentParser
from torchtext import data, datasets
from scipy.stats.stats import pearsonr
from scipy.special import expit as sigmoid


def load_model(file):
    model = torch.load(file, map_location=lambda storage, loc: storage)
    return model

def linearize(a, b, c, activation_fn):
    a_contrib = 0.5 * (activation_fn(a + b + c)  + activation_fn(a + c) - activation_fn(b + c) - activation_fn(c))
    b_contrib = 0.5 * (activation_fn(a + b + c)  - activation_fn(a + c) + activation_fn(b + c) - activation_fn(c))
    return a_contrib, b_contrib, activation_fn(c)

def linearize_tanh(a, b):
    a_contrib = 0.5 * (np.tanh(a) + np.tanh(a + b) - np.tanh(b)) 
    b_contrib = 0.5 * (np.tanh(b) + np.tanh(a + b) - np.tanh(a))
    return a_contrib, b_contrib

def CD(batch, model, start, stop):
    weights = model.lstm.state_dict()
    
    W_ii, W_if, W_ig, W_io = np.split(weights['weight_ih_l0'], 4, 0)
    W_hi, W_hf, W_hg, W_ho = np.split(weights['weight_hh_l0'], 4, 0)
    W_out = model.hidden2label.weight.data
    b_i, b_f, b_g, b_o = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 4)

    # The second axis is garbage, we remove it. Resulting matrix is of size (#words) x (length of glove vector, 300)
    word_vecs = model.word_embeddings(batch.text)[:,0].data
    #word_vecs = model.word_embeddings(review)[:,0].data
    
    L = word_vecs.size(0)
    phrase = np.zeros((L, model.hidden_dim))  #phrase contribution
    rest = np.zeros((L, model.hidden_dim))    #rest of the contribution
    phrase_h = np.zeros((L, model.hidden_dim))
    rest_h = np.zeros((L, model.hidden_dim))
    
    #iterate through word_vecs
    for i in range(L):
        if i == 0:
            #there is no prev
            prev_phrase_h = np.zeros(model.hidden_dim)
            prev_rest_h = np.zeros(model.hidden_dim)
        else:
            prev_phrase_h = phrase_h[i-1]
            prev_rest_h = rest_h[i-1]
            
        #calculating o, f, i, g    
        phrase_o = np.dot(W_ho, prev_phrase_h)
        phrase_f = np.dot(W_hf, prev_phrase_h)
        phrase_i = np.dot(W_hi, prev_phrase_h)
        phrase_g = np.dot(W_hg, prev_phrase_h)
        
        rest_o = np.dot(W_ho, prev_rest_h)
        rest_f = np.dot(W_hf, prev_rest_h)
        rest_i = np.dot(W_hi, prev_rest_h)
        rest_g = np.dot(W_hg, prev_rest_h)

        #only modify for range [start, stop]
        if (start <= i) and (i <= stop):
            phrase_o = phrase_o + np.dot(W_io, word_vecs[i])
            phrase_f = phrase_f + np.dot(W_if, word_vecs[i])
            phrase_i = phrase_i + np.dot(W_ii, word_vecs[i])
            phrase_g = phrase_g + np.dot(W_ig, word_vecs[i])
        else:
            rest_o = rest_o + np.dot(W_io, word_vecs[i])
            rest_f = rest_f + np.dot(W_if, word_vecs[i])
            rest_i = rest_i + np.dot(W_ii, word_vecs[i])
            rest_g = rest_g + np.dot(W_ig, word_vecs[i])
        
        #calculate contributions to i, g
        phrase_contrib_i, rest_contrib_i, bias_contrib_i = linearize(phrase_i, rest_i, b_i, sigmoid)
        phrase_contrib_g, rest_contrib_g, bias_contrib_g = linearize(phrase_g, rest_g, b_g, np.tanh)

        phrase[i] = phrase_contrib_i * (phrase_contrib_g + bias_contrib_g) + bias_contrib_i * phrase_contrib_g
        rest[i] = rest_contrib_i * (phrase_contrib_g + rest_contrib_g + bias_contrib_g) + (phrase_contrib_i + bias_contrib_i) * rest_contrib_g

        #add bias for range [start,stop)
        if i >= start and i < stop:
            phrase[i] += bias_contrib_i * bias_contrib_g
        else:
            rest[i] += bias_contrib_i * bias_contrib_g
        
        #When there's a prev, calculate contributions
        if i > 0:
            phrase_contrib_f, rest_contrib_f, bias_contrib_f = linearize(phrase_f, rest_f, b_f, sigmoid)
            phrase[i] += (phrase_contrib_f + bias_contrib_f) * phrase[i-1]
            rest[i] += (phrase_contrib_f + rest_contrib_f + bias_contrib_f) * rest[i-1] + rest_contrib_f * phrase[i-1]

        o = sigmoid(np.dot(W_io, word_vecs[i]) + np.dot(W_ho, prev_phrase_h + prev_rest_h) + b_o)
        phrase_contrib_o, rest_contrib_o, bias_contrib_o = linearize(phrase_o, rest_o, b_o, sigmoid)
        new_phrase_h, new_rest_h = linearize_tanh(phrase[i], rest[i])
        phrase_h[i] = o * new_phrase_h
        rest_h[i] = o * new_rest_h
    
    #calculating final scores
    phrase_scores = np.dot(W_out, phrase_h[L-1])
    rest_scores = np.dot(W_out, rest_h[L-1])

    return phrase_scores, rest_scores, phrase_h[L-1], rest_h[L-1]

def load_model(file):
    model = torch.load(file, map_location=lambda storage, loc: storage)
    return model

def get_batches(batch_nums, train_iterator, dev_iterator, dset='train'):
    print('Getting batches.')
    # pick data_iterator
    if dset=='train':
        data_iterator = train_iterator
    elif dset=='dev':
        data_iterator = dev_iterator
    #get batches
    num = 0
    batches = {}
    data_iterator.init_epoch() 
    for batch_idx, batch in enumerate(data_iterator):
        if batch_idx == batch_nums[num]:
            batches[batch_idx] = batch
            num +=1 

        if num == max(batch_nums):
            break
        elif num == len(batch_nums):
            print('found them all')
            break
    return batches

def get_sst():    
    inputs = data.Field(lower='preserve-case')
    answers = data.Field(sequential=False, unk_token=None)

    # build with subtrees so inputs are right
    train_s, dev_s, test_s = datasets.SST.splits(inputs, answers, fine_grained = False, train_subtrees = True,
                                           filter_pred=lambda ex: ex.label != 'neutral')
    inputs.build_vocab(train_s, dev_s, test_s)
    answers.build_vocab(train_s)
    
    # rebuild without subtrees to get longer sentences
    train, dev, test = datasets.SST.splits(inputs, answers, fine_grained = False, train_subtrees = False,
                                       filter_pred=lambda ex: ex.label != 'neutral')
    
    train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=1, device=-1)

    return inputs, answers, train_iter, dev_iter

def preprocessingBOW(batch,vocab,istrain):
    tensor = torch.FloatTensor(len(vocab),len(batch))
    tensor.zero_()
    
    for i in xrange(len(batch)):
        # update frequency
        c = Counter()
        c.update(batch[i])
        
        localtensor = torch.FloatTensor(len(vocab))
        localtensor.zero_()
        
        localtensor[c.keys()] = torch.FloatTensor(c.values())
        tensor[:,i] = localtensor
    
    return tensor

In [ ]:
inputs, answers, train_iterator, dev_iterator = get_sst()

In [ ]:
#get batches for train and dev set
n_dev = 872 
batch_num_dev = list(range(n_dev+1))
batches_dev = get_batches(batch_num_dev, train_iterator, dev_iterator, dset='dev')
n_train = 11000
batch_num_train = list(range((n_train+1)))
batches_train = get_batches(batch_num_train, train_iterator, dev_iterator, dset='train')

In [ ]:
print(n_train, len(batches_train))

In [ ]:
#load LSTM model and logistic regression weights
model_file = "model.pt"
LR_weights_file = "LR_weights.npy"

model = load_model(model_file)
LR_weights = np.load(LR_weights_file)

In [ ]:
inputsBOW = data.Field(lower='preserve-case', tensor_type=torch.FloatTensor, postprocessing=preprocessingBOW)

# No tokenization applied because the data is not seq
# unk_token=None: ignore out of vocabulary tokens, since these are grades
answersBOW = data.Field(sequential=False, unk_token=None)

# fine_grained=False - use the following grade mapping { 0,1 -> negativ; 2 -> neutral; 3,4 -> positive }
# filter=... - remove the neutral class to reduce the problem to binary classification
# train_subtrees=False - Use only complete review instead of also using subsentences (subtrees)
trainBOW, devBOW, testBOW = datasets.SST.splits(inputsBOW, answersBOW, fine_grained = False, train_subtrees = True,
                                       filter_pred=lambda ex: ex.label != 'neutral')
# build the initial vocabulary from the SST dataset
inputsBOW.build_vocab(trainBOW, devBOW, testBOW)

# build the vocab for the labels (only consists of 'positive','negative')
answersBOW.build_vocab(trainBOW)

device=-1 #-1 for CPU and integer otherwise
vocab_size = len(inputsBOW.vocab)

trainBOW_iter, devBOW_iter, testBOW_iter = data.BucketIterator.splits(
        (trainBOW, devBOW, testBOW), repeat=False, batch_size=1, device=device)

batchesBOW = get_batches(batch_num_dev, trainBOW_iter, devBOW_iter, dset="dev") 

In [ ]:
LR_coeff = {}
text = batchesBOW[100].text.data
#determine logistic regression coefficient for each word in dev set
for i in range(n_dev):
    if (i % (100) == 0):
        print("%d out of %d" % (i, n_dev))
    text = batches_dev[i].text.data[:, 0]
    words = [inputs.vocab.itos[idx] for idx in text]
    L = len(words)
    LR_coeff[i] = np.zeros(L)
    for j in range(L):
        one_hot_idx = inputsBOW.vocab.stoi[words[j]]
        word_vec = np.zeros(vocab_size)
        word_vec[one_hot_idx] = 1
        coeff = np.dot(word_vec,LR_weights.T)
        LR_coeff[i][j] = coeff[0] - coeff[1]

In [ ]:
CD_scores = {}
#determine CD score for each word in dev set
for i in range(n_dev):
    if (i % (100) == 0):
        print("%d out of %d" % (i, n_dev))
    batch = batches_dev[i]
    word_vecs = model.word_embeddings(batch.text)[:,0].data
    L = word_vecs.size(0)
    CD_scores[i] = np.zeros(L)
    for j in range(L):
        phrase_score, _, _, _ = CD(batch, model, j, j)
        CD_scores[i][j] = phrase_score[0] - phrase_score[1]

In [ ]:
x_LR = np.array([])
y_CD = np.array([])
#generate numpy arrays for LR coefficients and CD score, 
#such that each point x, y correspond to a word in dev set
for i in range(n_dev):
    for j in range(len(CD_scores[i])):
        x_LR = np.append(x_LR, LR_coeff[i][j])
        y_CD = np.append(y_CD, CD_scores[i][j])

In [ ]:
#calculate pearson correlation coefficient r
(r, p_value) = pearsonr(x_LR, y_CD)

#calculating least squares regression line
A = np.vstack([x_LR, np.ones(len(x_LR))]).T
m, c = np.linalg.lstsq(A, y_CD)[0]

#plotting CD scores vs LR coeffcients 
fig1 = plt.figure(1)
plt.scatter(x_LR, y_CD, color='g', s=0.2)
#also plot best fit line
plt.plot(x_LR, m*x_LR + c, linewidth=1)
plt.xlabel("Logistic Regression Coeffcients")
plt.ylabel("Contextual Decomposition scores")
fig1.savefig("CDvsLR_unigrams.png")

In [ ]:
print("Pearson correlation for SST:", r)

In [ ]:
CD_embeddings_train = {}
#calculate average embedding in train data set, for every unigram and bigram in train set 
for i in range(n_train):
    if (i % (100) == 0):
        print("%d out of %d" % (i, n_train))
    batch = batches_train[i]
    text = batch.text.data[:, 0]
    words = [inputs.vocab.itos[i] for i in text]
    word_vecs = model.word_embeddings(batch.text)[:,0].data
    L = len(words)
    for j in range(L):
        #unigrams
        current = words[j]
        _, _, CD_embed, _ = CD(batch, model, j, j)
        if current in CD_embeddings_train:
            k = CD_embeddings_train[current][0] + 1
            CD_embeddings_train[current] = [k, CD_embeddings_train[current][1] * float(k-1)/k + CD_embed/float(k)]
        else:
            CD_embeddings_train[current] = [1, CD_embed]
        #bigrams
        if j < (L-1):
            current = words[j] + " " + words[j+1]
            _, _, CD_embed, _ = CD(batch, model, j, j+1)
            if current in CD_embeddings_train:
                k = CD_embeddings_train[current][0] + 1
                CD_embeddings_train[current] = [k, CD_embeddings_train[current][1] * float(k-1)/k + CD_embed/float(k)]
            else:
                CD_embeddings_train[current] = [1, CD_embed]  

In [ ]:
import copy

CD_embeddings = copy.deepcopy(CD_embeddings_train)

for i in range(n_dev):
    if (i % (100) == 0):
        print("%d out of %d" % (i, n_dev))
    batch = batches_dev[i]
    text = batch.text.data[:, 0]
    words = [inputs.vocab.itos[i] for i in text]
    word_vecs = model.word_embeddings(batch.text)[:,0].data
    L = len(words)
    for j in range(L):
        #unigrams
        current = words[j]
        _, _, CD_embed, _ = CD(batch, model, j, j)
        if current in CD_embeddings:
            k = CD_embeddings[current][0] + 1
            CD_embeddings[current] = [k, CD_embeddings[current][1] * float(k-1)/k + CD_embed/float(k)]
        else:
            CD_embeddings[current] = [1, CD_embed]
        #bigrams
        if j < (L-1):
            current = words[j] + " " + words[j+1]
            _, _, CD_embed, _ = CD(batch, model, j, j+1)
            if current in CD_embeddings:
                k = CD_embeddings[current][0] + 1
                CD_embeddings[current] = [k, CD_embeddings[current][1] * float(k-1)/k + CD_embed/float(k)]
            else:
                CD_embeddings[current] = [1, CD_embed]  

In [ ]:
from sklearn.neighbors import NearestNeighbors
#nearest neighbors model
nearestNeighbors_model = NearestNeighbors(n_neighbors=20, metric='cosine')
freq_embeddings = CD_embeddings.values()
k_embeddings = len(freq_embeddings[0][1])
n_embeddings = len(freq_embeddings)

embeddings = np.zeros((n_embeddings, k_embeddings))
ngram_embeddings = []

for ngram, embedding in CD_embeddings.iteritems():
    ngram_embeddings.append([ngram, embedding[1]])
for i in range(n_embeddings):
    embeddings[i] = freq_embeddings[i][1]
#fit knn model to embeddings
nearestNeighbors_model = nearestNeighbors_model.fit(embeddings)

In [ ]:
#calculate nearest neighbours for selected phrases
selected_phrases = ['bad', 'not bad', 'not funny', 'very funny', 'entertaining'] 
selected_embeddings = np.zeros((len(selected_phrases), k_embeddings))
for i in range(len(selected_phrases)):
    selected_embeddings[i] = CD_embeddings[selected_phrases[i]][1]
    
selected_nearestNeighbours = nearestNeighbors_model.kneighbors(selected_embeddings, return_distance=False)
print(selected_nearestNeighbours)

In [ ]:
for i in range(len(selected_nearestNeighbours)):
    print("Next:")
    for j in range(len(selected_nearestNeighbours[i])):
        print(ngram_embeddings[selected_nearestNeighbours[i][j]][0])